In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import numpy as np 
import pandas as pd 
import shutil
import os
import zipfile
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
import torch.nn.functional as F
import copy
import tqdm
import time
from PIL import Image

import albumentations
from albumentations import pytorch as AT

from efficientnet_pytorch import EfficientNet

%matplotlib inline

In [ ]:
train_dir = '../input/plant-pathology-2021-224x224/train_imgs'
test_dir = '../input/plant-pathology-2021-fgvc8/test_images'

In [ ]:
train_data = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
a = np.asarray(train_data['labels'].unique())

In [ ]:
len(a)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()
train_data['num_labels'] = encoder.fit_transform(train_data['labels'])

In [ ]:
train_data

In [ ]:
train_files = os.listdir(train_dir)
test_f = os.listdir(test_dir)

In [ ]:
test_files = pd.DataFrame()
test_files['image'] = test_f
test_files

In [ ]:
train_files = train_data
train_files

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, file_csv, dir, transform=None, mode = 'train'):
        self.file_csv = file_csv
        self.dir = dir
        self.transform = transform
        self.mode = mode
            
    def __len__(self):
        return len(self.file_csv)
    
    #метод который позволяет нам индексировать датасет
    def __getitem__(self, idx):
        #считываем изображение
        image = cv2.imread(os.path.join(self.dir, self.file_csv['image'][idx]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        self.label = int(train_data['num_labels'][idx])
        
        #применяем аугментации
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        if self.mode == 'train':
            return image, float(self.label)
        else:
            return image, self.file_csv['image'][idx]


In [ ]:
batch_size = 16
num_workers = 0
img_size = 224

In [ ]:
data_transforms_train = albumentations.Compose([
    albumentations.Resize(img_size, img_size),
    AT.ToTensor()
    ])

data_transforms_test = albumentations.Compose([
    albumentations.Resize(img_size, img_size),
    AT.ToTensor()
    ])

In [ ]:
trainset = CustomDataset(train_files, train_dir, transform = data_transforms_train)
testset = CustomDataset(test_files, test_dir, transform=data_transforms_test, mode = 'test')

In [ ]:
valid_size = int(len(train_files) * 0.1)
train_set, valid_set = torch.utils.data.random_split(trainset, 
                                    (len(train_files)-valid_size, valid_size))

In [ ]:
len(train_set), len(valid_set)

In [ ]:
trainloader = torch.utils.data.DataLoader(train_set, pin_memory=True, 
                                        batch_size=batch_size, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_set, pin_memory=True, 
                                        batch_size=batch_size, shuffle=True)

testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size,
                                         num_workers = num_workers)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b3')

In [ ]:
model

In [ ]:
model._fc = nn.Linear(in_features = 1536, out_features = 12)

In [ ]:
def train_model(model_conv, train_loader, valid_loader, criterion, optimizer,scheduler ,n_epochs):
    model_conv.to(device)
    valid_loss_min = np.Inf
    patience = 5
    # сколько эпох ждем до отключения
    p = 0
    # иначе останавливаем обучение
    stop = False

    # количество эпох
    for epoch in range(1, n_epochs+1):
        print(time.ctime(), 'Epoch:', epoch)

        train_loss = []

        for batch_i, (data, target) in enumerate(train_loader):

            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model_conv(data)
            loss = criterion(output, target.long())
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
    # запускаем валидацию
        model_conv.eval()
        correct = 0
        val_loss = []
        for batch_i, (data, target) in enumerate(valid_loader):
            data, target = data.to(device), target.to(device)
            output = model_conv(data)
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == target).sum().item()
            loss = criterion(output, target.long())
            val_loss.append(loss.item()) 
        
        acc = correct / len(valid_set)

        print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}.')
        print(f'Accuracy on valid set: {acc}')

        valid_loss = np.mean(val_loss)
        scheduler.step(valid_loss)
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model_conv.state_dict(), 'model.pt')
            valid_loss_min = valid_loss
            p = 0

        # проверяем как дела на валидации
        if valid_loss > valid_loss_min:
            p += 1
            print(f'{p} epochs of increasing val loss')
            if p > patience:
                print('Stopping training')
                stop = True
                break        

        if stop:
            break
    return model_conv, train_loss, val_loss

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.8, patience=2,)

In [ ]:
# model_, train_loss, val_loss = train_model(model, trainloader, validloader, criterion, 
#                               optimizer,scheduler, n_epochs=15)

In [ ]:
# pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b3')
model._fc = nn.Linear(in_features = 1536, out_features = 12)

model.state_dict(torch.load('./model.pt'))

In [ ]:
from pathlib import Path

model.to(device)
model.eval()
df_preds = pd.DataFrame()
for path in Path('../input/plant-pathology-2021-fgvc8/test_images').iterdir():
    img = cv2.imread(str(path))[:, ::-1]
    img = data_transforms_test(image=img)['image'].cuda()
    pred = model(img[None])
    
    df_preds = df_preds.append(
        {'image': path.parts[-1], 'labels': encoder.inverse_transform(torch.argmax(pred.cpu(), dim=1))[0]},
        ignore_index=True)
    
df_preds

In [ ]:
df_preds.to_csv('./submission.csv', index = False)